In [1]:
%cd ..

/nfs/scistore16/edelgrp/fzimin/depth-poset


/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/ipykern

AttributeError: _ARRAY_API not found

# Utils

In [3]:
def load_text(path):
    # read the file
    with open(path, 'r') as file:
        text = file.read()
    return text

In [4]:
def get_line_starts_with(s, start):
    s = s.split('\n')
    for line in s:
        if line.find(start) == 0:
            return line

## Exclude info functions

In [5]:
def get_error_line(log):
    # returns an error line if exists
    for s in log.split('\n'):
        if s.find('error') != -1:
            return s

In [6]:
def get_dim(log):
    for s in log.split('\n'):
        if s.find('dim=') == 0:
            return s[4:]

In [7]:
def get_n(log):
    for s in log.split('\n'):
        if s.find('n=') == 0:
            return s[2:]

In [8]:
def get_error(line):
    if line is None:
        return 'no errors'
    if line.find('out-of-memory') != -1:
        return 'out-of-memory'
    if line.find('the following arguments are required'):
        return "input-error"
    return line

# Depth Poset Scores

## Logs

In [9]:
# path to log-files
path_logs_scores = 'logs/scores_on_extended_cubical_torus'

# define path for each log-file
df_logs_scores = pd.DataFrame({'log-path': os.listdir(path_logs_scores)})
df_logs_scores['log-path'] = df_logs_scores['log-path'].apply(lambda s: os.path.join(path_logs_scores, s))
df_logs_scores = df_logs_scores[df_logs_scores['log-path'].apply(lambda s: Path(s).suffix) == '.log']

# define job index
df_logs_scores['job'] = df_logs_scores['log-path'].apply(lambda s: s[36:-4])

# load logs
df_logs_scores['log'] = df_logs_scores['log-path'].apply(load_text)

# get params from log
df_logs_scores['dim'] = df_logs_scores['log'].apply(get_dim)
df_logs_scores['n'] = df_logs_scores['log'].apply(get_n)

df_logs_scores['error_line'] = df_logs_scores['log'].apply(get_error_line)
df_logs_scores['error'] = df_logs_scores['error_line'].apply(get_error)


print(f'df_logs_scores.shape = {df_logs_scores.shape}')
df_logs_scores.head()

df_logs_scores.shape = (632, 7)


,log-path,job,log,dim,n,error_line,error
0,logs/scores_on_extended_cubical_torus/output_4...,s/output_45485165,SLURM_ARRAY_TASK_ID = '141'\nPARAMS:\ndim=1\nn...,1,56,None,no errors
1,logs/scores_on_extended_cubical_torus/output_4...,s/output_45484344,SLURM_ARRAY_TASK_ID = '26'\nPARAMS:\ndim=1\nn=...,1,14,None,no errors
2,logs/scores_on_extended_cubical_torus/output_4...,s/output_45489215,SLURM_ARRAY_TASK_ID = '386'\nPARAMS:\ndim=2\nn...,2,48,None,no errors
3,logs/scores_on_extended_cubical_torus/output_4...,s/output_45484395,SLURM_ARRAY_TASK_ID = '77'\nPARAMS:\ndim=1\nn=...,1,28,None,no errors
4,logs/scores_on_extended_cubical_torus/output_4...,s/output_45490681,SLURM_ARRAY_TASK_ID = '533'\nPARAMS:\ndim=3\nn...,3,16,slurmstepd-eta357: error: Detected 1 oom-kill ...,out-of-memory


In [12]:
df_errors = df_logs_scores[['dim', 'n', 'error']].value_counts()
df_errors = df_errors.reset_index()
df_errors = df_errors.pivot_table(columns=['dim', 'error'], index='n', values='count')
df_errors = df_errors.fillna(0).astype('Int32')

def sort_key(index):
    index = index.strip()
    return (0, 0) if index == '' else (1, int(index))
df_errors = df_errors.loc[sorted(df_errors.index, key=sort_key)]

df_errors

dim                       1           2                   3              
error input-error no errors input-error no errors no errors out-of-memory
n                                                                        
                2         0           0         0         0             0
3               0        10           0        10        10             0
4               0        10           0        10        10             0
5               0        10           0        10        10             0
6               0         0           0         0        10             0
8               0         0           0         0        10             0
12              0        10           0        10        10             0
13              0        10           0        10        10             0
14              0        10           0        10        10             0
15              0        10           0        10        10             0
16              0        10           0        10         0            10
20              0        10           0        10         0             0
24              0        10           0        10         0             0
28              0        10           5         5         0             0
32              0        10           0        10         0             0
36              0        10           0        10         0             0
40              0        10           0        10         0             0
44              0        10           0        10         0             0
48              0        10           0        10         0             0
52              0        10           0        10         0             0
56              0        10           0        10         0             0
60              0        10           0        10         0             0
64              0        10           0        10         0             0
68              0        10           0        10         0             0
72              0        10           0        10         0             0
76              0        10           0        10         0             0
80              0        10           0        10         0             0
84              0        10           0         0         0             0
88              0        10           0         0         0             0
92              0        10           0         0         0             0
96              0        10           0         0         0             0
100             0        10           0         0         0             0